In [2]:
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout


import keras
from keras.layers import Embedding
from keras.layers import Dense, Flatten, LSTM
from keras.layers import Input, GlobalMaxPool1D, Dropout
from keras.layers import Activation
from keras.models import Model, Sequential
from keras import optimizers

In [3]:
print(tf.__version__)

2.8.0


In [3]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

In [4]:
df1 = pd.read_csv('datasets/geo/data/full_dataset/goemotions_1.csv')

In [5]:
df1.shape

(70000, 37)

In [6]:
df2 = pd.read_csv('datasets/geo/data/full_dataset/goemotions_2.csv')

In [7]:
df2.shape

(70000, 37)

In [8]:
df3 = pd.read_csv('datasets/geo/data/full_dataset/goemotions_3.csv')

In [9]:
df3.shape

(71225, 37)

In [10]:
#concatenate the 3 datasets

In [11]:
df = pd.concat([df1, df2, df3], axis=0)

In [12]:
df.shape

(211225, 37)

In [13]:
df.head(3)

,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,admiration,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,eew5j0j,Brdd9,nrl,t3_ajis4z,t1_eew18eq,1.548381e+09,1,False,0,...,0,0,0,0,0,0,0,1,0,0
1,>sexuality shouldn’t be a grouping category I...,eemcysk,TheGreen888,unpopularopinion,t3_ai4q37,t3_ai4q37,1.548084e+09,37,True,0,...,0,0,0,0,0,0,0,0,0,0
2,"You do right, if you don't care then fuck 'em!",ed2mah1,Labalool,confessions,t3_abru74,t1_ed2m7g7,1.546428e+09,37,False,0,...,0,0,0,0,0,0,0,0,0,1


In [14]:
#Delete unwanted columns

In [15]:
df.drop(['created_utc','id', 'author', 'subreddit', 'link_id', 'parent_id', 
         'rater_id', 'example_very_unclear'], 
        inplace=True, axis=1)

In [16]:
df.head(3)

,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,>sexuality shouldn’t be a grouping category I...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"You do right, if you don't care then fuck 'em!",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [17]:
# available emotions
emotions = []
for col in df.columns:
    emotions.append(col)
    
emotions_df = pd.DataFrame(emotions)

In [18]:
emotions_df.head()

,0
0,text
1,admiration
2,amusement
3,anger
4,annoyance


In [19]:
import string
pucn = string.punctuation
stop = stopwords.words('english')

In [20]:
def clean_sentences(sentence):
    url = re.compile(r'https?://\S+|www\.\S+')
    url_free = url.sub(r'', sentence)
    alpha = ''.join([word for word in url_free if not word.isdigit()])
    no_white_space = re.sub(r'\s+', ' ',  alpha)  
    no_non_words = re.sub('\w*\d\w*', '', no_white_space)
    html=re.compile(r'<.*?>')
    no_html = html.sub(r'',no_non_words)
    no_punct=[words for words in no_html if words not in pucn]
    words_wo_punct=''.join(no_punct)
    words_wo_punct = words_wo_punct.lower()
    emoji = re.compile("["
        u"\U0001F600-\U0001F64F" 
        u"\U0001F300-\U0001F5FF"  
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF" 
        u"\U00002500-\U00002BEF"  
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030""]+", re.UNICODE)
    no_emoji = re.sub(emoji, '', words_wo_punct)
    return no_emoji

In [21]:
comments = list(df['text'])

In [22]:
len(comments)

211225

In [23]:
test_sentence = comments[200]

In [24]:
test_sentence

"Something's wrong. The top post on r/conspiracy isn't completely anti-conspiracy? Wow, feels weird. /s(?)"

In [25]:
cleaned_text = clean_sentences(test_sentence)

In [26]:
cleaned_text

'somethings wrong the top post on rconspiracy isnt completely anticonspiracy wow feels weird s'

In [27]:
X =[]
for sentence  in comments:
    X.append(clean_sentences(sentence))

In [28]:
X[200]

'somethings wrong the top post on rconspiracy isnt completely anticonspiracy wow feels weird s'

In [29]:
emotions

['text',
 'admiration',
 'amusement',
 'anger',
 'annoyance',
 'approval',
 'caring',
 'confusion',
 'curiosity',
 'desire',
 'disappointment',
 'disapproval',
 'disgust',
 'embarrassment',
 'excitement',
 'fear',
 'gratitude',
 'grief',
 'joy',
 'love',
 'nervousness',
 'optimism',
 'pride',
 'realization',
 'relief',
 'remorse',
 'sadness',
 'surprise',
 'neutral']

In [30]:
len(emotions)

29

In [31]:
y = df[[
 'admiration',
 'amusement',
 'anger',
 'annoyance',
 'approval',
 'caring',
 'confusion',
 'curiosity',
 'desire',
 'disappointment',
 'disapproval',
 'disgust',
 'embarrassment',
 'excitement',
 'fear',
 'gratitude',
 'grief',
 'joy',
 'love',
 'nervousness',
 'optimism',
 'pride',
 'realization',
 'relief',
 'remorse',
 'sadness',
 'surprise',
 'neutral']].values

In [32]:
y

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [33]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, random_state=42)

In [39]:
tokenizer = Tokenizer() 
tokenizer.fit_on_texts(X_train) # create word to index relationship-each word is given a unique index mostly according to appearance in the corpus

vocab_size = len(tokenizer.word_index) + 1 #size of the total number of uniques tokens in the dataset

tokenized_train = tokenizer.texts_to_sequences(X_train) #converting tokens into sequence of integers
tokenized_test = tokenizer.texts_to_sequences(X_test)

In [40]:
X_train[10]

'you forgot the to be sure  its a surprise to be sure back into the hole with you '

In [42]:
tokenized_train[10]

[5, 491, 1, 3, 19, 133, 30, 4, 837, 3, 19, 133, 115, 157, 1, 1659, 26, 5]

In [43]:
max_len = 100

padded_train = pad_sequences(tokenized_train, maxlen=max_len, padding='pre') 

padded_test = pad_sequences(tokenized_test, maxlen=max_len, padding='pre')

In [44]:
padded_train[10]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    5,  491,    1,    3,   19,  133,
         30,    4,  837,    3,   19,  133,  115,  157,    1, 1659,   26,
          5], dtype=int32)

In [45]:
y[10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1])

In [46]:
vocab_size #all unique tokens in the corpus

32746

In [47]:

"""
create an embedding matrix with glove words  and vectors 
- each word is a key and a corresponding vector is the value
"""
emmbeddings_dictionary = {}

glove_model =  open('glove.6B.100d.txt', encoding='utf8')
    
for line in glove_model:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:],'float32')
    emmbeddings_dictionary[word]=vector
    
glove_model.close()    

In [48]:
"""
create an embedding matrix for the corpus where each word in the corpus is a key and the corresponding vector 
of a dimension of 100 is the value
each row contains a number which corresponds to the index of the word in the corpus
the matrix has 100 columns-each column contains word embedding for each word in the corpus
"""

embedding_dim=100
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, index in tokenizer.word_index.items():
    embedding_vector = emmbeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index]=embedding_vector


In [49]:
embedding_matrix.shape

(32746, 100)

In [50]:
model = Sequential() #initialize sequential model

# create embedding layer
embedding_layer = Embedding(vocab_size, embedding_dim,
                            weights=[embedding_matrix], input_length=max_len, trainable=False)

# add layers to the model
model.add(embedding_layer)
model.add(LSTM(256, return_sequences = True))
# model.add(Dropout(0.2))
model.add(LSTM(128))
# model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(28, activation='sigmoid'))

In [51]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          3274600   
                                                                 
 lstm (LSTM)                 (None, 100, 256)          365568    
                                                                 
 lstm_1 (LSTM)               (None, 128)               197120    
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 28)                1820      
                                                                 
Total params: 3,847,364
Trainable params: 572,764
Non-trainable params: 3,274,600
_________________________________________________________________


In [52]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [ ]:
history=model.fit(padded_train, y_train, epochs=20, batch_size=128, validation_split=0.3)

Epoch 1/20
809/809 [==============================] - 2039s 3s/step - loss: 0.1619 - accuracy: 0.2581 - val_loss: 0.1582 - val_accuracy: 0.2588
Epoch 2/20
809/809 [==============================] - 2033s 3s/step - loss: 0.1533 - accuracy: 0.2747 - val_loss: 0.1455 - val_accuracy: 0.2948
Epoch 3/20
809/809 [==============================] - 2044s 3s/step - loss: 0.1370 - accuracy: 0.3392 - val_loss: 0.1332 - val_accuracy: 0.3581
Epoch 4/20
809/809 [==============================] - 2033s 3s/step - loss: 0.1299 - accuracy: 0.3679 - val_loss: 0.1291 - val_accuracy: 0.3692
Epoch 5/20
809/809 [==============================] - 2037s 3s/step - loss: 0.1263 - accuracy: 0.3787 - val_loss: 0.1269 - val_accuracy: 0.3718
Epoch 6/20
809/809 [==============================] - 2039s 3s/step - loss: 0.1241 - accuracy: 0.3861 - val_loss: 0.1254 - val_accuracy: 0.3764
Epoch 7/20
809/809 [==============================] - 2028s 3s/step - loss: 0.1222 - accuracy: 0.3935 - val_loss: 0.1246 - val_accuracy:

In [ ]:
y_pred = model.predict(padded_test)

In [ ]:
model.save('EmoText')